## Distributed Model Training

### University of Virginia
### DS 7200: Distributed Computing
### Last Updated: October 17, 2025

---

### SOURCES: 

- [Distributed Machine Learning Frameworks and its Benefits](https://www.xenonstack.com/blog/distributed-ml-framework#:~:text=In%20distributed%20machine%20learning%2C%20model,and%20training%20each%20split%20separately.)

- [Distributed Training with Azure](https://learn.microsoft.com/en-us/azure/machine-learning/concept-distributed-training?view=azureml-api-2/)

- [Distributed Training: Guide for Data Scientists
](https://neptune.ai/blog/distributed-training)

- Mastering Reinforcement Learning with Python, Enes Bilgin.

- [Distributed model training II: Parameter Server and AllReduce](http://www.juyang.co/distributed-model-training-ii-parameter-server-and-allreduce/)

- [Distributed Machine Learning and the Parameter Server](https://www.cs.cornell.edu/courses/cs4787/2019sp/notes/lecture22.pdf)


### OBJECTIVES

- Explain approaches for distributed model training
- Identify the benefits and challenges of synchronous and asynchronous training
- Explain the parameter server and Allreduce algorithms
- Explain why Ring-Allreduce can work better than Allreduce 

### CONCEPTS

- Data parallelism and model parallelism
- Asynchronous training vs Synchronous training 
- Parameter server algorithm
- Allreduce algorithm and Ring-Allreduce algorithm
- Vertical partitioning and horizontal partitioning of a model

---

### I. Why use Distributed Model Training?


For massive training sets, it may not be possible to train a model on a single machine. 

This may be the case for deep learning models.

In *distributed training*, the workload to train a model is split up and shared among worker nodes. 

The concepts, benefits, and challenges are similar to what we've learned earlier.

The work can be parallelized to speed up training.

This process introduces benefits but also complexity.

---

### II. Data Parallelism and Model Parallelism

Two main types of distributed training: *data parallelism* and *model parallelism*.

#### Data Parallelism

This follows the approach used by Spark

Data is divided into partitions

Number of partitions = total number of available nodes

Model is copied in each worker node

Each node operates on its subset of data



<img src="./img/data_parallelism.png" width=600>

Each node must:

- Independently compute errors between training sample predictions and labels
- Update its model based on errors
- Communicate all of its changes to the other nodes to update their corresponding models

Worker nodes need to synchronize gradients at end of batch computation to ensure they're training a consistent model.

---

#### Model Parallelism

In some cases, the model may be too large for a worker

The strategy is to segment the model into different parts that run concurrently in different workers

Each model part runs on same data

Scalability depends on degree of task parallelization of algorithm

Worker nodes need to synchronize shared parameters, usually once for each forward or backward-propagation step 

More complex to implement than data parallelism

**Example of model parallelized on two GPUs**


<img src="./img/model_parallelism.png" width=400>

Model may be divided horizontally or vertically into different parts 

The parts run concurrently in different workers with each worker running on the same data

Worker needs to synchronize shared parameters, usually once for each forward or backward-propagation step

This **illustration** shows different partitions. Vertical partitioning keeps together all neurons in a layer, which works well for training deep learning models.  

<img src="./img/horiz_vert_part.png" width=400>

---

### III. Synchronous Training

Consider data parallelism case: 
- Data is divided into partitions
- Each partition is sent to a worker 
- Each worker has full replica of model. Training is done on its partition. 

**Forward Pass**

In synchronous training, forward pass begins at same time for each worker

Each worker computes different output

**Backward Pass**

Each worker performs backward pass and has local gradient (since trained on different data subsets):

<img src="./img/loss.png" width=150>

To synchronize across workers, gradients are aggregated (averaging is most common):

<img src="./img/avg_grad.png" width=150>

Model parameters are updated using averaged gradient:

<img src="./img/gradient_descent.png" width=150>

This ensures all workers have synched parameters

But how to efficiently aggregate?

---

#### Quick Background: Reduce vs Allreduce

We discussed **reduce**, as in MapReduce:

Compute a reduction (generally a sum) of data on multiple machines $M_1, M_2, ..., M_n$ and materialize result on one machine $D$

The difference with **Allreduce** is the destination of the result: it is materialized on the machines $M_1, M_2, ..., M_n$




---

#### Synchronous Algorithm Example: Allreduce Algorithm

Each node has a subset of the data 

Each node does a calculation (e.g., model gradients) and distributes to the other nodes

**Very communication heavy**

Algorithm reduces the target arrays in all workers to a single array and returns the resultant array to all workers

Gradients are collected across workers, averaged, used to update model weights on each node

Operation proceeds elementwise.

*Illustration where values are aggregated elementwise*

<img src="./img/allreduce.png" width=600>

source: https://tech.preferred.jp/en/blog/technologies-behind-distributed-deep-learning-allreduce/


There are different implementations:

**1. Naive Allreduce**

Each worker sends gradients to single worker called the *driver*.  

The driver reduces the gradients and sends updated gradients to all workers.  

**Challenge with this approach**: the driver is a bottleneck.  
Each node must ship gradients to the driver, and the driver must send updated gradients back to each node.  
This scales linearly with $N$.  
As the number of nodes increases, communication cost and reduction step scales poorly.


<img src="./img/single_gpu_reducer.png" width=600>

Source [here](https://andrew.gibiansky.com/blog/machine-learning/baidu-allreduce/#:~:text=The%20GPUs%20in%20a%20ring,data%20from%20its%20left%20neighbor.&text=The%20algorithm%20proceeds%20in%20two,%2C%20and%20then%2C%20an%20allgather.)


---

**2. Ring-Allreduce** 

This is a very clever approach that is strategic about the data structure and algorithm.

Results in an algorithm that runs in **constant time.**

The intuition is that each GPU does a fraction of the work and passes the result to its right neighbor

It also receives a fraction of work from its left neighbor

Workers are set up in a logical ring

---

*Benefits in Neural Networks*

Each worker is in charge of subset of parameters (weights) shared only with the next worker in the ring  

Less time spent sending data and more time spent doing computations on data locally on each GPU  

Allows for efficient averaging of gradients in neural networks across many devices and many nodes  

Retains the determinism and predictable convergence properties of synchronous stochastic gradient descent

<img src="./img/ringreduce.png" width=600>

**How it Works**

The algorithm consists of two steps:

- Scatter-reduce: GPUs exchange data such that each GPU ends up with chunk of final result.
- Allgather: GPUs exchange chunks such that all GPUs end up with complete final result.

**For an elementwise addition task, we start with this:**

<img src="./img/start.png" width=600>

**We end with this:**

<img src="./img/end.png" width=600>

Now we demonstrate *scatter-reduce*

On each iteration, each GPU sends one chunk to neighbor on right and receives one chunk from neighbor on left

The data is accumulated on each node and pushed around the ring like this:


<img src="./img/iteration1.png" width=600>

<img src="./img/iteration2.png" width=600>

After $N-1$ iterations, each node will hold the sum of all values across GPUs for a chunk (see below).

Each worker does some of the reduction which eases workload

This [post](https://andrew.gibiansky.com/blog/machine-learning/baidu-allreduce/#:~:text=The%20GPUs%20in%20a%20ring,data%20from%20its%20left%20neighbor.&text=The%20algorithm%20proceeds%20in%20two,%2C%20and%20then%2C%20an%20allgather.) has a nice illustration and helpful details.

<img src="./img/scatter_reduce_final.png" width=600>

*Allgather*

This step is very similar to scatter-reduce and runs for $N-1$ iterations

Instead of accumulating values, the final sum is copied to the neighbor and overwrites its partial sum.

<img src="./img/allgather.png" width=600>


*Communication Cost*

Each of $N$ GPUs will send and receive values $N-1$ times for scatter-reduce, and $N-1$ times for allgather. 

On each iteration, GPUs send $\frac{K}{N}$ values, where $K$ = number of values in array being summed across different GPUs. 

Total data transferred to and from each GPU is then $\frac{2 (N - 1) K}{N}$

For large $N$, this is independent of $N$, which allows the algorithm to scale

*General Computation*

We looked at illustration with elementwise addition

Also applies for more complex calcs such as gradient aggregation

---

### IV. Asynchronous Training

We apply data parallelism here as well

Goal: Allows workers to work independently (not waiting on other workers)

Asynchronous training can be more efficient than synchronous training since there is no waiting. 

This is especially helpful when there is variation in the computing power across workers.

One way to achieve this is by using a parameter server.

#### Asynchronous Algorithm Example: Parameter Server

In this approach, weights and biases of ML model are distributed across nodes in cluster  

A copy of the model is stored on each node and a centralized *parameter server* manages model changes.  
This pattern is called *centralized training*.

Here is how it works:

1. Designate some nodes as parameter servers, while others train the model

- Parameter servers store model parameters, update global state of model

2. Training workers each get a subset of data
3. Each training worker fetches parameters from parameter servers
4. Each training worker performs training loop,  
   calculates gradients and loss,  
   sends gradients back to all parameter servers, which updates model parameters




<img src="./img/parameter_server.png" width=300>

**Challenges with this approach**

At any given time, only one worker is using updated version of model. Others are using stale version. 

If one worker is used as parameter server, it can be a bottleneck and single point of failure.  
Using *multiple parameter servers* alleviates this issue but adds communication costs.


---

**Parameter Server Example: Gorila - General Reinforcement Learning Architecture**

*Source: Mastering Reinforcement Learning with Python, Enes Bilgin.*

Background:  
- Deep Reinforcement Learning (DRL) requires massive training datasets to train a neural network. There is no label used; instead, a reward function is specified.
- The *Q-network* is one approach in DRL.
- An agent exists in some state (e.g., an investor holds a portfolio of stocks) and must take a series of actions over time (buy/sell orders).
- Everything outside of the agent is the environment (the economy, the stock markets)
- We might have historical data (experiences) that can be used
- The goal is to learn the best action to take given each state. This is the *optimal policy*. It needs to maximize long-term reward.

Strategy: we need to simulate experiences, and we can parallelize this. We store experiences in a replay buffer.

Here are the important components:

**Actors:**  
Processes that interact with copy of environment given a policy, take the action, observe the reward, and transition to the next state.

Actors get a copy of the Q-network from a parameter server

**Replay Buffer:**  
Actors collect experiences and store them in a replay buffer. This can be done in a distributed fashion.

**Learners:**  
Learners calculate the gradients that update the Q-network in the parameter server.  
A learner has a copy of the Q-network, it samples experiences from the buffer, calculates the loss and gradients, and sends them back to the parameter server

**Parameter Server:**  
The parameter server stores the main copy of the Q-network. As learning progresses, updates happen here.    
All processes sync their version of the Q-network from here.

--

**Next, we provide an overview of Gorila**

Gorila provides a general framework to parallelize deep Q-learning

It consists of bundles which can be distributed to workers, where each bundle comprises an action, learner, and local replay buffer.  

Next we show an illustration of the approach. For our purposes, the details are unimportant; the takeaway is how the parameter server supports learning.

<img src="./img/gorila.png" width=600>

Notice how the learners send gradients back to the parameter server

The learners get the synced Q-network from the parameter server

**Shortcoming:** There is a lot of passing around of parameters between the actors, learners, and parameter server.  
This can be a significant communication load.

If you are interested, you can [explore](https://arxiv.org/pdf/1803.00933) how the *Ape-X* architecture improves Gorila!



---

### V. Implementation

**Elephas** is a Keras add-on that allows you to use Spark to execute distributed deep learning models at scale.

**Horovod**. Open-source distributed training framework developed at Uber for TensorFlow, Keras, PyTorch, and MXNet.  
Supports practical distributed training over several GPUs and nodes.  
Documentation can be found [here](https://horovod.readthedocs.io/en/stable/)


---

### VI. Conclusions

We have studied different approaches for distributed model training

Benefits can include scalability, efficiency, and fault tolerance 

These come at the cost of increased network communication load

Data parallelism (partitioning the data across nodes) is easier than model parallelism

Model parallelism is useful when the model won't fit on a single machine. 

Vertical partitioning is easier for deep learning, as layers are kept together on workers

There are several packages that implement distributed model training

**Tradeoffs**

As always, there are tradeoffs to the methods:

- Centralized training, such as the parameter server approach, can introduce a communication bottleneck. More parameter servers can be added to reduce the bottleneck.

- The parameter server approach is asynchronous which can increase efficiency: there is no waiting for workers. This is especially helpful when the nodes have differences in processing power.


---